In [ ]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import glob
import os

In [ ]:
dir = os.chdir("/content/drive/MyDrive/BBC News Summary")
print(f'Directory change to {os.getcwd()}')

Directory change to /content/drive/MyDrive/BBC News Summary


In [ ]:
def import_data(folders_n, def_path):
  filename = [path + "/News Articles/" + folder + "/" + "*.txt"
     for path,folder in zip([def_path]*len(folders_n), folders_n)]
  temp_doc = []
  print(filename)
  # tags 
  for x in filename:
    data = glob.glob(x)
    for y in data:
      with open(y, encoding="latin-1") as f:
        topic = x.split('/')[len(x.split('/'))-2]
        # print('me',topic)
        line = f.readlines()
        # head = line[0].strip()
        body = ''.join([l.strip() for l in line[1:]])
        temp_doc.append([topic,body])
    print(f"Loading data from \033[1m{topic}\033[0m directory")
    print("\nEntire Data is loaded successfully")

  return temp_doc

In [ ]:
folders_n = ['business','entertainment','politics','sport','tech']
doc = import_data(folders_n = folders_n, def_path = os.getcwd())
# docs = load_data(folder_names = folders_n, root_path = os.getcwd())


['/content/drive/MyDrive/BBC News Summary/News Articles/business/*.txt', '/content/drive/MyDrive/BBC News Summary/News Articles/entertainment/*.txt', '/content/drive/MyDrive/BBC News Summary/News Articles/politics/*.txt', '/content/drive/MyDrive/BBC News Summary/News Articles/sport/*.txt', '/content/drive/MyDrive/BBC News Summary/News Articles/tech/*.txt']
Loading data from business directory

Entire Data is loaded successfully
Loading data from entertainment directory

Entire Data is loaded successfully
Loading data from politics directory

Entire Data is loaded successfully
Loading data from sport directory

Entire Data is loaded successfully
Loading data from tech directory

Entire Data is loaded successfully


In [ ]:
print(len(doc))
doc1 = pd.DataFrame(doc, columns = ['category','text'])
length = len(doc1)
# print(length)
doc1.head(length)
# print(doc1)

2225


,category,text
0,business,Billions of farmers' livelihoods are at risk f...
1,business,"Circuit City Stores, the second-largest electr..."
2,business,Ask Jeeves has become the third leading online...
3,business,Japan's economy teetered on the brink of a tec...
4,business,China's efforts to stop the economy from overh...
...,...,...
2220,tech,"Gambling is hugely popular, especially with te..."
2221,tech,A group of artists in Poland has taken the cac...
2222,tech,The number of personal computers worldwide is ...
2223,tech,The proportion of surfers using Microsoft's In...


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

# download necessary resources for NLTK
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    text = ' '.join(tokens)
    return text

doc1['text_processed'] = doc1['text'].apply(preprocess_text)
print(doc1['text'])


0       Billions of farmers' livelihoods are at risk f...
1       Circuit City Stores, the second-largest electr...
2       Ask Jeeves has become the third leading online...
3       Japan's economy teetered on the brink of a tec...
4       China's efforts to stop the economy from overh...
                              ...                        
2220    Gambling is hugely popular, especially with te...
2221    A group of artists in Poland has taken the cac...
2222    The number of personal computers worldwide is ...
2223    The proportion of surfers using Microsoft's In...
2224    The 'bad behaviour' of e-mail users is helping...
Name: text, Length: 2225, dtype: object


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import accuracy_score
df = doc1

train_size = int(len(df) * 0.8)
train_data = df[:train_size]
test_data = df[train_size:]

# Extract features using TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(train_data['text'])
X_test = vectorizer.transform(test_data['text'])
y_train = train_data['category']
y_test = test_data['category']

# Train the SVM model
model = svm.SVC(kernel='sigmoid')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.09887640449438202
